In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [6]:
with cProfile.Profile() as pr:
  rec = await data_source.load(data, cache_with_db=True)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\base.py:1967 in       │
│ _exec_single_context                                                                             │
│                                                                                                  │
│   1964 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1965 │   │   │   │   │   │   │   break                                                         │
│   1966 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1967 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1968 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1969 │   │   │   │   │   )                                                                     │
│   1970                                                                                           │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\default.py:941 in     │
│ do_execute                                                                                       │
│                                                                                                  │
│    938 │   │   cursor.executemany(statement, parameters)                                         │
│    939 │                                                                                         │
│    940 │   def do_execute(self, cursor, statement, parameters, context=None):                    │
│ ❱  941 │   │   cursor.execute(statement, parameters)                                             │
│    942 │                                                                                         │
│    943 │   def do_execute_no_params(self, cursor, statement, context=None):                      │
│    944 │   │   cursor.execute(statement)                                                         │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\duckdb_engine\__init__.py:140 in        │
│ execute                                                                                          │
│                                                                                                  │
│   137 │   │   │   elif parameters is None:                                                       │
│   138 │   │   │   │   self.__c.execute(statement)                                                │
│   139 │   │   │   else:                                                                          │
│ ❱ 140 │   │   │   │   self.__c.execute(statement, parameters)                                    │
│   141 │   │   except RuntimeError as e:                                                          │
│   142 │   │   │   if e.args[0].startswith("Not implemented Error"):                              │
│   143 │   │   │   │   raise NotImplementedError(*e.args) from e                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ParserException: Parser Error: Table must have at least one column!

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│ ❱ 2   rec = await dat

In [7]:
rec

{'term': 'test', 'result_count': 3, 'results': [{'number': 2, 'name': 'fixing cars', 'start': '2020-01-03', 'end': '2020-01-06', 'status': 'started', 'org': {'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown', 'projects': <LoadStatus.unloaded: 1>, '_id': UUID('45ac0c3b-f2af-479a-a25f-510832af9c52')}, 'tasks': [{'name': 'task6', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'John', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('2e9ac474-8057-4879-91b0-3d8409e1542c')}], 'status': 'todo', '_id': UUID('b66e8c75-1caa-4d56-9f8d-221cad236fd8')}], 'members': [{'name': 'John', 'age': 20, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('cd6545d7-dcb3-4bf3-91f9-09cc5516ae86')}, {'name': 'Jane', 'age': 30, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('9ed6a653-20ee-4997-bfe9-506cd60cf760')}, {'name': 'Jack', 'age': 40, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('ccc99041-030f-4ff3-942f-7b9e32c1ad84')}]}, {'number': 1, 'name': 'cle

In [8]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         544143 function calls (463446 primitive calls) in 0.641 seconds

   Ordered by: cumulative time
   List reduced from 858 to 72 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       67    0.001    0.000    0.637    0.010 C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:433(_load_record)
       62    0.000    0.000    0.568    0.009 C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:495(load_record)
      388    0.000    0.000    0.535    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1831(__hash__)
43242/388    0.069    0.000    0.534    0.001 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:39(gen_int_hash)
12029/4408    0.005    0.000    0.520    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(<genexpr>)
10635/4020    0.005    0.000    0.517    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:33(_hash_sequence)
20396/80